In [6]:
import pandas as pd
import re

# Load the Excel file
file_path = 'booking.xlsx'

# Define cleaning functions
def clean_price(price):
    # Remove all non-numeric characters (e.g., 'MAD', spaces)
    return re.sub(r'[^\d]', '', str(price))

def remove_spaces(text):
    # Remove extra spaces
    return re.sub(r'\s{2,}', ' ', str(text))

def clean_ra_la_lo(ra_la_lo):
    ra_la_lo = re.sub(r'[^\d.,]', '', str(ra_la_lo))
    # Remove extra spaces
    return re.sub(r',', '.', str(ra_la_lo))

def clean_column(df, col):
    # If column contains 'price' in the name, clean it using clean_price
    if 'price' in col.lower():
        return df[col].apply(clean_price)

    elif 'latiude' in col.lower() or 'longitude' in col.lower() or 'rating' in col.lower():
        return df[col].apply(clean_ra_la_lo)
    else:
        return df[col]
    
df = pd.read_excel(file_path)


# Apply the cleaning function to all columns
for col in df.columns:
    df[col] = clean_column(df, col).apply(remove_spaces)

# Save the cleaned data back to an Excel file
cleaned_file_path = 'cleaned_booking.xlsx'
df.to_excel(cleaned_file_path, index=False)

print(f"Cleaned file saved to: {cleaned_file_path}")


Cleaned file saved to: cleaned_booking.xlsx


In [4]:
from utils.db import get_database_connection
from collections import defaultdict

from flask import Flask, render_template, request, jsonify
from threading import Thread
import time
import requests
import time
import re
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from bs4 import BeautifulSoup
import mysql.connector

global tables
tables = ['booking_scraped_urls','airbnb_scraped_urls']

def get_database_connection():
    try:
        conn = mysql.connector.connect(
            host=os.getenv('DB_HOST'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            database=os.getenv('DB_NAME')
        )
        print("Connection to the database was successful.")
        return conn
    except mysql.connector.Error as err:
        print(f"Error connecting to database: {err}")
        return None


def create_table_airbnb(conn):
    try:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS airbnb_scraped_urls (
                id INT AUTO_INCREMENT PRIMARY KEY,
                serial TEXT UNIQUE NOT NULL
                url VARCHAR(255) UNIQUE NOT NULL
            )
        ''')
        conn.commit()
        print("Tables `airbnb_scraped_urls` created successfully.")
        cursor.close()
    except mysql.connector.Error as err:
        print(f"Error creating tables ==> {err}")

def insert_url_airbnb(conn, url, ID):
    try:
        cursor = conn.cursor(buffer = True)
        ID = ID.strip()
        ID = str(ID)
        cursor.execute("INSERT INTO airbnb_scraped_urls (url, serial) VALUES (%s, %s)", (url, ID))
        conn.commit()
        print(f"Inserted URL with ID ==> {ID}")
        cursor.close()
    except mysql.connector.Error as err:
        print(f"Error inserting airbnb ID: {ID}")

def check_url_exists_airbnb(conn, ID):
        cursor = conn.cursor()
        ID = ID.strip()
        ID = str(ID)
        cursor.execute("SELECT 1 FROM airbnb_scraped_urls WHERE serial = %s", (ID,))
        result = cursor.fetchone()
        cursor.close()
        return result is not None
        



def save_to_xlsx_airbnb(listing_data, output_filename):
    # Check if the file exists, and load existing data if it does
    if os.path.exists(output_filename):
        df = pd.read_excel(output_filename)
    else:
        # Define the DataFrame with the appropriate columns
        df = pd.DataFrame(columns=['title', 'price', 'description', 'host', 'composition', 'rating', 'latitude', 'longitude', 'photo_links'])

    # Ensure listing_data is a list of dictionaries or list of lists
    if isinstance(listing_data, dict):  # If it's a single dictionary, wrap it in a list
        listing_data = [listing_data]

    # Convert the listing data into a DataFrame
    df_new = pd.DataFrame(listing_data)
    for col in df_new.columns:
        df_new[col] = clean_column(df_new, col).apply(remove_spaces)
        
    
    # Concatenate the new listings with the existing ones
    df = pd.concat([df, df_new], ignore_index=True)
    
    # Save the DataFrame to the Excel file
    df.to_excel(output_filename, index=False)
    print(f"Listings saved to {output_filename}")


def scrape_listing_details_airbnb(driver, url):
    driver.get(url)
    time.sleep(2)  # Adjust the wait time as needed

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract listing details (use the actual class names found from the inspection)
    title = soup.find('h1', class_='hpipapi atm_7l_1kw7nm4 atm_c8_1x4eueo atm_cs_1kw7nm4 atm_g3_1kw7nm4 atm_gi_idpfg4 atm_l8_idpfg4 atm_kd_idpfg4_pfnrn2 i1pmzyw7 atm_9s_1nu9bjl dir dir-ltr').text.strip() if soup.find('h1', class_='hpipapi atm_7l_1kw7nm4 atm_c8_1x4eueo atm_cs_1kw7nm4 atm_g3_1kw7nm4 atm_gi_idpfg4 atm_l8_idpfg4 atm_kd_idpfg4_pfnrn2 i1pmzyw7 atm_9s_1nu9bjl dir dir-ltr') else 'mkynch'
    price = soup.find('span', class_='_11jcbg2').text.strip() if soup.find('span', class_='_11jcbg2') else 'mkynch'
    description = soup.find('div', class_='d1isfkwk atm_vv_1jtmq4 atm_w4_1hnarqo dir dir-ltr').text.strip() if soup.find('div', class_='d1isfkwk atm_vv_1jtmq4 atm_w4_1hnarqo dir dir-ltr') else 'mkynch'
    host = soup.find('div', class_='t1pxe1a4').text.strip() if soup.find('div', class_='t1pxe1a4') else 'mkynch'
    composition = soup.find('ol', class_='lgx66tx atm_gi_idpfg4 atm_l8_idpfg4 dir dir-ltr').text.strip() if soup.find('ol', class_='lgx66tx atm_gi_idpfg4 atm_l8_idpfg4 dir dir-ltr') else 'mkynch'
    rating = soup.find('span', class_='_10nhpq7').text.strip() if soup.find('span', class_='_10nhpq7') else 'mkynch'
    r = requests.get(url)
    p_lat = re.compile(r'"lat":([-0-9.]+),')
    p_lng = re.compile(r'"lng":([-0-9.]+),')
    latitude = p_lat.findall(r.text)[0]
    longitude = p_lng.findall(r.text)[0]



    photo_links = [img['src'] for img in soup.find_all('img', class_='itu7ddv atm_e2_idpfg4 atm_vy_idpfg4 atm_mk_stnw88 atm_e2_1osqo2v__1lzdix4 atm_vy_1osqo2v__1lzdix4 i1cqnm0r atm_jp_pyzg9w atm_jr_nyqth1 i1de1kle atm_vh_yfq0k3 dir dir-ltr')]
    photo_links = ','.join(photo_links)
    
    return {
        'title': title,
        'price': price,
        'description': description,
        'host': host,
        'composition': composition,
        'rating': rating,
        'latitude': latitude,
        'longitude': longitude,
        'photo_links': photo_links
    }


scraping_thread = None
scraping_active_airbnb = True
scraping_active_booking = True
total_announcement_counts = 0
airbnb_announcement_counts = 0
booking_announcement_counts = 0

cities = ["rabat"]
conn = get_database_connection()
listings_airbnb_by_city = {city: [] for city in cities}
listings_booking_by_city = {city: [] for city in cities}

def scrape_airbnb_urls_edge_airbnb(cities, conn):
    global scraping_active_airbnb, total_announcement_counts, airbnb_announcement_counts
    edge_options = Options()
    # edge_options.add_argument("--headless")  # Run in headless mode (no GUI)
    # Initialize the Edge WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Edge(service=service, options=edge_options)
    global all_links
    all_links = []  # List to store all extracted links

    for city in cities:
        if not scraping_active_airbnb:
            break
        print(f"Scraping for {city}")
        base_url = f"https://www.airbnb.com/s/{city}/homes"
        sum = 0
        while base_url and scraping_active_airbnb:
            if not scraping_active_airbnb:
                break
            driver.get(base_url)
            
            # Wait for JavaScript to load content
            time.sleep(3)  # Adjust the wait time as needed

            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Find and process the listing links
            link_elements = soup.find_all('a', href=True, class_='l1ovpqvx atm_1he2i46_1k8pnbi_10saat9 atm_yxpdqi_1pv6nv4_10saat9 atm_1a0hdzc_w1h1e8_10saat9 atm_2bu6ew_929bqk_10saat9 atm_12oyo1u_73u7pn_10saat9 atm_fiaz40_1etamxe_10saat9 bn2bl2p atm_5j_223wjw atm_9s_1ulexfb atm_e2_1osqo2v atm_fq_idpfg4 atm_mk_stnw88 atm_tk_idpfg4 atm_vy_1osqo2v atm_26_1j28jx2 atm_3f_glywfm atm_kd_glywfm atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_uc_aaiy6o_1w3cfyq_oggzyc atm_70_1b8lkes_1w3cfyq_oggzyc atm_uc_glywfm_1w3cfyq_pynvjw atm_uc_aaiy6o_pfnrn2_ivgyl9 atm_70_1b8lkes_pfnrn2_ivgyl9 atm_uc_glywfm_pfnrn2_61fwbc dir dir-ltr')

            if not link_elements:
                print("No listings found. The class name may have changed or there are no listings available.")
            else:
                print(f"Found {len(link_elements)} listings on this page. Extracting URLs...")
                sum += len(link_elements)
                
                for link in link_elements:
                    if not scraping_active_airbnb:
                        break
                    url = "https://www.airbnb.com" + link['href']
                    part_after_slash = link['href'].split("/")[-1]
                    ID = part_after_slash.split("?")[0] 
                    
                    # Add the link and ID to the all_links list
                    all_links.append((url, ID))

            # Check and click the "Next" button
            next_button = soup.find('a', attrs={"aria-label": "hhh"})
            
            if next_button:
                next_url = next_button.get('href')
                base_url = "https://www.airbnb.com" + next_url
                print(f"Navigating to next page:")
                time.sleep(2)
            else:
                base_url = None
                print("No more pages to scrape.")
        print(f"{sum} announcements scraped")
        print(f"Finished scraping URLs for {city}")

    # Process all the links after scraping all cities
    for url, ID in all_links:
        if not scraping_active_airbnb:
            break
        print(f"Processing URL: {url}")
        time.sleep(1)
        if not check_url_exists_airbnb(conn, ID):
            insert_url_airbnb(conn, url, ID)
            total_announcement_counts += 1
            airbnb_announcement_counts += 1
            listing_data = scrape_listing_details_airbnb(driver, url)
            output_filename = f"test_airbnb.xlsx"  # Assuming the same output file for all cities
            save_to_xlsx_airbnb(listing_data, output_filename)
        else:
            print(f"ID already exists: {ID}")


    driver.quit()
    return all_links


def get_all_links():
    return all_links
    


Connection to the database was successful.


In [5]:
scrape_airbnb_urls_edge_airbnb(cities, conn)

Scraping for agadir
Found 18 listings on this page. Extracting URLs...
No more pages to scrape.
18 announcements scraped
Finished scraping URLs for agadir
Processing URL: https://www.airbnb.com/rooms/1195165478582729878?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-09-14&check_out=2024-09-19&source_impression_id=p3_1724001661_P3xri7Nqu925Awev&previous_page_section_name=1000&federated_search_id=3b8ee6d0-b564-4c26-9b14-2d5f0a72e123


InternalError: Unread result found

In [1]:
from utils.db import get_database_connection
from collections import defaultdict

from flask import Flask, render_template, request, jsonify
from threading import Thread
import time
import requests
import time
import re
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from bs4 import BeautifulSoup
import mysql.connector

global tables
tables = ['booking_scraped_urls','airbnb_scraped_urls']

def get_database_connection():
    try:
        conn = mysql.connector.connect(
            host=os.getenv('DB_HOST'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            database=os.getenv('DB_NAME')
        )
        print("Connection to the database was successful.")
        return conn
    except mysql.connector.Error as err:
        print(f"Error connecting to database: {err}")
        return None


def create_table_booking(conn):
    try:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS booking_scraped_urls (
                id INT AUTO_INCREMENT PRIMARY KEY,
                serial TEXT UNIQUE NOT NULL
                url VARCHAR(255) UNIQUE NOT NULL
            )
        ''')
        conn.commit()
        print("Tables `booking_scraped_urls` created successfully.")
    except mysql.connector.Error as err:
        print(f"Error creating tables: {err}")

def insert_url_booking(conn, url, ID):
    try:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO booking_scraped_urls (url, serial) VALUES (%s, %s)", (url, ID))
        conn.commit()
        print(f"Inserted URL with ID:: {ID}")
    except mysql.connector.Error as err:
        print(f"Error inserting URL: {err}")
    finally:
        cursor.close()

def check_url_exists_booking(conn, url):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT 1 FROM booking_scraped_urls WHERE url = %s", (url,))
        return cursor.fetchone() is not None
    except mysql.connector.Error as err:
        print(f"Error checking URL existence")
        return False


def save_to_xlsx_booking(listing_data, output_filename):
    if os.path.exists(output_filename):
        df = pd.read_excel(output_filename)
    else:
        df = pd.DataFrame(columns=['url', 'title', 'price', 'description', 'host', 'composition', 'rating', 'latitude', 'longitude', 'photo_links'])

    df_new = pd.DataFrame([listing_data])
    df = pd.concat([df, df_new], ignore_index=True)
    df.to_excel(output_filename, index=False)
    print(f"Listing saved to {output_filename}")


def scrape_listing_details_booking(driver, url):
    driver.get(url)
    time.sleep(2)  # Adjust the wait time as needed

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract listing details (use the actual class names found from the inspection)
    title = soup.find('h2', class_='af32860db5 pp-header__title').text.strip() if soup.find('h2', class_='af32860db5 pp-header__title') else 'mkynch'
    # First, find the td element
    price = soup.find('span', class_='prco-valign-middle-helper')
    if price:
        price = price.text.strip()
    else:
        price = 0.0

    description = soup.find('p', class_='e2585683de c8d1788c8c').text.strip() if soup.find('p', class_='e2585683de c8d1788c8c') else 'mkynch'
    host = soup.find('div', class_='t1pxe1a4').text.strip() if soup.find('div', class_='t1pxe1a4') else 'mkynch'
    composition = soup.find('div', class_='hprt-roomtype-bed').text.strip() if soup.find('div', class_='hprt-roomtype-bed') else 'mkynch'
    rating = soup.find('div', class_='d0522b0cca fd44f541d8').find('div', class_='a447b19dfd').next_sibling.strip() if soup.find('div', class_='d0522b0cca fd44f541d8') else 0.0
    latitude = "mkynch"
    longitude = "mkynch"
    map_element = soup.find('a', class_="loc_block_link_underline_fix")
    if map_element:
        if "data-atlas-latlng" in map_element.attrs:
            coordinates = map_element["data-atlas-latlng"]
            # Split the coordinates by comma
            coords = coordinates.split(',')
            latitude, longitude = coords
    photo_links = []

    try:
        # Open the target URL
        driver.get(url)
        
        try:
            # Find and click the link to load more photos
            photos_link = driver.find_element(By.CSS_SELECTOR, 'a.bh-photo-grid-item.bh-photo-grid-thumb.js-bh-photo-grid-item-see-all')
            photos_link.click()
        except Exception as e:
            print(f"Error finding or clicking the photo link: {e}")
            return photo_links  # Return the initialized but empty list
        
        # Wait for the photos to load
        time.sleep(2)

        try:
            # Get the updated page source after the click
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            sources = ["src", "srcset", "data-srcset", "data-src"]
            img_elements = soup.find_all('img', class_='bh-photo-modal-grid-image')
            if img_elements:
                for img in img_elements:
                    for attr in sources:
                        if attr in img.attrs:
                            photo_links.append(img[attr])
                            break  
            
                # Convert the list to a comma-separated string
                photo_links = [','.join(photo_links)]
            else:
                print("No image elements found.")
        
        except Exception as e:
            print(f"Error processing images: {e}")

    except Exception as e:
        print(f"Error during the scraping process: {e}")

    print(photo_links)

    return {
        'url': url,
        'title': title,
        'price': price,
        'description': description,
        'host': host,
        'composition': composition,
        'rating': rating,
        'latitude': latitude,
        'longitude': longitude,
        'photo_links': photo_links
    }


scraping_thread = None
scraping_active_airbnb = True
scraping_active_booking = True
global total_announcement_counts, airbnb_announcement_counts, booking_announcement_counts
total_announcement_counts = 0
airbnb_announcement_counts = 0
booking_announcement_counts = 0

cities = ["agadir", "fes", "rabat", "casablanca"]
conn = get_database_connection()
listings_airbnb_by_city = {city: [] for city in cities}
listings_booking_by_city = {city: [] for city in cities}

def scrape_booking(cities, conn):
    global scraping_active_booking, total_url_counts, booking_url_counts, total_booking_url_counts
    booking_url_counts = 0
    total_booking_url_counts = 0
    total_url_counts = 0

    edge_options = Options()
    # edge_options.add_argument("--headless")  # Run in headless mode (no GUI)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Edge(service=service, options=edge_options)

    links = []

    for city in cities:
        if not scraping_active_booking:
            break
        print(f"Scraping for {city}")
        base_url = f"https://www.booking.com/searchresults.fr.html?ss={city}&checkin=2024-09-01&checkout=2024-09-02&group_adults=1&no_rooms=1&group_children=0"
        sum = 0
        print(f"while base_url and scraping_active_booking. {city}.....")
        if not scraping_active_booking:
            break
        driver.get(base_url)
        time.sleep(5)
        # Scroll down and click "More results" to load all listings
        while True and scraping_active_booking:
            print("while True and scraping_active_booking......")
            last_height = driver.execute_script("return document.body.scrollHeight")
            # Scroll to the bottom of the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Adjust the wait time as needed

            # Check if the height has increased
            new_height = driver.execute_script("return document.body.scrollHeight")

            # If the height has increased, continue scrolling
            if new_height > last_height:
                last_height = new_height
                continue  # Go back to scrolling

            # If height hasn't increased, look for the "More results" button
            try:
                more_results_button = driver.find_element(By.CSS_SELECTOR, 'button.hhhhhdba1b3bddf.e99c25fd33.ea757ee64b.f1c8772a7d.ea220f5cdc.f870aa1234')
                if more_results_button.is_displayed():  # Ensure the button is visible
                    more_results_button.click()
                    time.sleep(3)  # Adjust the wait time as needed
                    last_height = driver.execute_script("return document.body.scrollHeight")  # Update height after clicking
                else:
                    print("No 'More results' button found.")
                    break  # Exit the loop if button is not found or not clickable
            except Exception as e:
                print(f"No more results to load or error occurred:")
                break  # Exit the loop if an exception occurs
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Try clicking the "more results" button to load more listings

            
        # Find and process the listing links
        link_elements = soup.find_all('h3', class_='d3e8e3d21a')
        for link_element in link_elements:
                if not scraping_active_booking:
                    break
                links.append(link_element.find('a', href=True))
        print(f"Found {len(link_elements)} for {city}...")


    if not links:
        print("No listings found. The class name may have changed or there are no listings available.")
    else:
        total_booking_url_counts = len(links)
        print(f"Found {len(links)} listings on this page. Extracting URLs...")
        sum += len(links)
        output_filename = f"test_{city}_booking.xlsx"

        regex = r"sr_pri_blocks=([^&]+)"
        for link in links:
            if not scraping_active_booking:
                break
            url = link['href']
            print(f"Found URL: {url}")
            match = re.search(regex, link["href"])
            if match:
                sr_pri_blocks_id = match.group(1)
                print(f"Found ID: {sr_pri_blocks_id}")
            else:
                print("not found match")


            if not check_url_exists_booking(conn, sr_pri_blocks_id):
                insert_url_booking(conn, url, sr_pri_blocks_id)
                total_url_counts += 1
                booking_url_counts += 1
                listing_data = scrape_listing_details_booking(driver, url)
                save_to_xlsx_booking(listing_data, output_filename)
            else:
                print(f"URL already exists: {url}")

        '''for link in links:
            save_to_xlsx_booking(listing_data, output_filename)  '''
            
    print(sum)
    print("Finished scraping and inserting URLs.")
    driver.quit()  


Connection to the database was successful.


In [2]:

scrape_booking(cities, conn)



Scraping for agadir
while base_url and scraping_active_booking. agadir.....
while True and scraping_active_booking......
while True and scraping_active_booking......
while True and scraping_active_booking......
No more results to load or error occurred:
Found 75 for agadir...
Scraping for fes
while base_url and scraping_active_booking. fes.....
while True and scraping_active_booking......
while True and scraping_active_booking......
while True and scraping_active_booking......
No more results to load or error occurred:
Found 75 for fes...
Scraping for rabat
while base_url and scraping_active_booking. rabat.....
while True and scraping_active_booking......
while True and scraping_active_booking......
while True and scraping_active_booking......
No more results to load or error occurred:
Found 75 for rabat...
Scraping for casablanca
while base_url and scraping_active_booking. casablanca.....
while True and scraping_active_booking......
while True and scraping_active_booking......
while T

KeyboardInterrupt: 

In [30]:
import pandas as pd
import re

# Load the Excel file
file_path = 'booking.xlsx'

# Define cleaning functions
def clean_price(price):
    # Remove all non-numeric characters (e.g., 'MAD', spaces)
    return re.sub(r'[^\d]', '', str(price))

def remove_spaces(text):
    # Remove extra spaces
    return re.sub(r'\s{2,}', ' ', str(text))

def clean_ra_la_lo(ra_la_lo):
    ra_la_lo = re.sub(r'[^\d.,]', '', str(ra_la_lo))
    # Remove extra spaces
    return re.sub(r',', '.', str(ra_la_lo))

def clean_column(df, col):
    # If column contains 'price' in the name, clean it using clean_price
    if 'price' in col.lower():
        return df[col].apply(clean_price)

    elif 'latiude' in col.lower() or 'longitude' in col.lower() or 'rating' in col.lower():
        return df[col].apply(clean_ra_la_lo)
    else:
        return df[col]
    
df = pd.read_excel(file_path)


# Apply the cleaning function to all columns
for col in df.columns:
    df[col] = clean_column(df, col).apply(remove_spaces)

# Save the cleaned data back to an Excel file
cleaned_file_path = 'cleaned_booking.xlsx'
df.to_excel(cleaned_file_path, index=False)

print(f"Cleaned file saved to: {cleaned_file_path}")


Cleaned file saved to: cleaned_booking.xlsx
